In [1]:
#https://buomsoo-kim.github.io/keras/2018/04/24/Easy-deep-learning-with-Keras-5.md/

In [2]:
# 케라스에 내장된 mnist 데이터 셋을 함수로 불러와 바로 활용 가능하다
from keras.datasets import mnist
import matplotlib.pyplot as plt

(X_train, y_train), (X_test, y_test) = mnist.load_data()

Using TensorFlow backend.


11493376/11490434 [==============================] - 2s 0us/step


In [10]:
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split

# reshaping X data: (n, 28, 28) => (n, 784)
X_train = X_train.reshape((X_train.shape[0], -1))
X_test = X_test.reshape((X_test.shape[0], -1))

# 학습 과정을 단축시키기 위해 학습 데이터의 1/3만 활용한다
X_train, _ , y_train, _ = train_test_split(X_train, y_train, test_size = 0.67, random_state = 7)

# 타겟 변수를 one-hot encoding 한다
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(6534, 784) (10000, 784) (6534, 10) (10000, 10)


In [13]:
import numpy as np
y_train = np.argmax(y_train, axis = 1)
y_test = np.argmax(y_test, axis = 1)

print(y_train.shape)

(6534,)


In [5]:
# from keras.models import Sequential
# from keras.layers import Activation, Dense, BatchNormalization
# from keras import optimizers


# def make_model(dense):
#   #N = len(white_wine)
#   n_in=11
#   n_hiddens=32
#   n_out=11
#   p_keep=0.5 # 드롭아웃 확률의 비율

#   model = Sequential()

#   model.add(Dense(dense,input_dim=n_in))
#   model.add(BatchNormalization())
#   model.add(Activation('relu'))
#   model.add(Dense(dense))
#   model.add(BatchNormalization())
#   model.add(Activation('relu'))
#   model.add(Dense(dense))
#   model.add(BatchNormalization())
#   model.add(Activation('relu'))
#   model.add(Dense(dense))
#   model.add(BatchNormalization())
#   model.add(Activation('relu'))
#   model.add(Dense(units=n_out,activation='softmax'))

#   sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
#   #model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#   model.compile(loss='sparse_categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#   return model

# epochs = 100
# batch_size =150

# # 서로 다른 모델을 만들어 합치기 (Model Ensemble)
# model1 = KerasClassifier(build_fn=make_model(500),epochs=epochs, verbose=0)
# model1._estimator_type="classifier" #https://hororolol.tistory.com/362
# model2 = KerasClassifier(build_fn=make_model(100),epochs=epochs, verbose=0)
# model2._estimator_type="classifier"
# model3 = KerasClassifier(build_fn=make_model(50),epochs=epochs, verbose=0)
# model3._estimator_type="classifier"

# ensemble_model=VotingClassifier(estimators = [('model1', model1), ('model2', model2), ('model3', model3)], voting = 'soft')

In [17]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score
# #from skmultilearn.ensemble import MajorityVotingClassifier

def mlp_model():
    model = Sequential()
    
    model.add(Dense(50, input_shape = (784, )))
    model.add(Activation('sigmoid'))    
    model.add(Dense(50))
    model.add(Activation('sigmoid'))    
    model.add(Dense(50))
    model.add(Activation('sigmoid'))    
    model.add(Dense(50))
    model.add(Activation('sigmoid'))    
    model.add(Dense(10))
    model.add(Activation('softmax'))
    
    sgd = optimizers.SGD(lr = 0.001)
    model.compile(optimizer = sgd, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
    return model

model1 = KerasClassifier(build_fn = mlp_model, epochs = 100, verbose = 0)
model1._estimator_type="classifier"
model2 = KerasClassifier(build_fn = mlp_model, epochs = 100, verbose = 0)
model2._estimator_type="classifier"
model3 = KerasClassifier(build_fn = mlp_model, epochs = 100, verbose = 0)
model3._estimator_type="classifier"

ensemble_clf = VotingClassifier(estimators = [('model1', model1), ('model2', model2), ('model3', model3)], voting = 'soft')

In [7]:
#muilti-label???
#http://scikit.ml/api/skmultilearn.ensemble.voting.html
#!pip install scikit-multilearn

In [8]:
#ensemble_model= EnsembleVoteClassifier(clfs=[model1,model2,model3],weights=[1,1,1],voting='soft')

# from skmultilearn.ensemble import MajorityVotingClassifier
# from skmultilearn.cluster import FixedLabelSpaceClusterer
# from skmultilearn.problem_transform import ClassifierChain
# from sklearn.naive_bayes import GaussianNB

# classifier = MajorityVotingClassifier(
#     clusterer = FixedLabelSpaceClusterer(clusters = [[1,2,3], [0, 2, 5], [4, 5]]),
#     classifier = ClassifierChain(classifier=GaussianNB())
# )
# classifier.fit(X_train,y_train)
# predictions = classifier.predict(X_test)

In [18]:
#single label...
ensemble_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('model1',
                              <keras.wrappers.scikit_learn.KerasClassifier object at 0x7fb9e09f9358>),
                             ('model2',
                              <keras.wrappers.scikit_learn.KerasClassifier object at 0x7fb9e09f96a0>),
                             ('model3',
                              <keras.wrappers.scikit_learn.KerasClassifier object at 0x7fb9e09f9518>)],
                 flatten_transform=True, n_jobs=None, voting='soft',
                 weights=None)

In [19]:
y_pred = ensemble_clf.predict(X_test)
print(y_pred)

[1 1 1 ... 1 1 1]


In [20]:
print('Test accuracy:', accuracy_score(y_pred, y_test))

Test accuracy: 0.1135
